---
        
연구주제: h.p. lovecraft 단편 소설 sna 분석
   
코드 구성:
단편 하나를 문장단위로 끊어서 공출현 행렬을 만드는 함수 text2csv_of_co()

단편 여러개에 대해 코사인 유사도 분석을 하는 함수 texts2csv_of_cossim()

그리고 단편을 읽어와 두 함수를 적용하는 부분으로 나뉩니다.


데이터셋 링크: <https://data.world/mattgawarecki/hp-lovecraft/workspace/project-summary?agentid=mattgawarecki&datasetid=hp-lovecraft>

코사인 유사도 분석은 <https://wikidocs.net/book/2155>에 있는 코드를 썼습니다.

---

datetime: 2021/8/9 17:18:56 
수정1: text2csv_of_co함수 주석 변경
      stopwords 사용자 추가
      text2csv_of_co함수에서 표제어 추출 TextBlob 안되는 거 확인하고 NLTK로 바꿈    

In [1]:
# !pip install nltk
# !pip install textblob
# !pip install sklearn
# !pip install numpy
# !pip install pandas
# !pip install tqdm

In [2]:
from textblob import TextBlob

import nltk
import nltk.tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import linear_kernel

from collections import Counter

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jinhajin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def text2csv_of_co(lines: str, name: str):
    lines_textblob = TextBlob(lines).lower()
    
    #문장 나누기
    lines_made_sentence_with_textblob = [x.words for x in lines_textblob.sentences]
    lines_sentence = [' '.join(list(x)) for x in lines_made_sentence_with_textblob]
    # lines_sentence
    
    #토큰화
    list_of_sentences_words = []
    for line in lines_sentence:
        text_tokens = word_tokenize(line)
        a_sentence_words = [word for word in text_tokens if not word in stopwords.words()]
        list_of_sentences_words.append(a_sentence_words)
    # list_of_sentences_words
    
    
    #표제어 추출
    temp_list = []
    for a_sentence_words in list_of_sentences_words:
        temp = [WordNetLemmatizer().lemmatize(x, pos='v') for x in a_sentence_words]
        temp2 = [''.join(x) for x in temp] #WordList타입을 str로 바꾸기 위한 코드
        temp_list.append(temp2)
    words_list = temp_list    #words_list가 최종 정리된 문장 리스트. 2차원 배열이고 lines_textblob.sentences()에 맞춰서 문장이 나눠져있음
    
    
    #핵심어 정하기 위한 단어 빈도 세기
    tokens = []
    for a_sentence_words in list_of_sentences_words:
        tokens.extend(a_sentence_words)
    counted_tokens = Counter(tokens)
    most_list = counted_tokens.most_common()
    
    
    #본문에 3번이상 나온 단어를 핵심어로 정함
    keywords_list = [string for string, fre in most_list if fre > 2] #(전처리한)문서 내 3번 이상 등장하는 단어만 핵심어.
                                                                 # 이 리스트를 써서 csv를 만들거임
    
    #공출현 행렬 변수
    df = pd.DataFrame(0, index=keywords_list, columns=keywords_list) #키워드 대칭행렬 만들거임

    
#     for idx, word in enumerate(keywords_list): #대각선에는 문서 내 핵심어의 빈도
#         df[word][word] = most_list[idx][1]

        
    #문장단위 공출현 빈도 구하기
    for word in keywords_list:
        for line in lines_textblob.sentences:
            count = 0
            if line.find(word) != -1: #만약 핵심어 중 하나가 line 내에 있는 경우
                for word2 in keywords_list: #그 외 핵심어도 위 line 내에 있는지 확인하고 count를 올림
                    if word == word2:
                        pass
                    elif word != word2:
                        if line.find(word2) != -1:
                            df[word][word2] += 1
                        
    return df

In [4]:
def texts2csv_of_cossim(list_lines: list, list_name: list): #각 소설이 하나씩 들어가있는 1차원 배열
    pd_list = pd.DataFrame(list_lines)
    
    #토큰화
    pd_list[0] = pd_list.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
    
    #표제어 추출
    pd_list[0] = pd_list[0].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
#     print(pd_list.head(5))
    
    #길이 2 이하 단어 제거
    tokenized_doc = pd_list[0].apply(lambda x: [word for word in x if len(word) > 2])
#     print(tokenized_doc[:5])
    
    # 역토큰화 (토큰화 작업을 되돌림)
    detokenized_doc = []
    for i in range(len(pd_list)):
        t = ' '.join(tokenized_doc[i])
        detokenized_doc.append(t)
    
    # 다시 pd_list[0]에 재저장
    pd_list[0] = detokenized_doc 
    
    #sklearn 라이브러리 함수로 TF-IDF 행렬 만들기
    vectorizer = TfidfVectorizer(stop_words='english', 
    max_features= 1000) # 상위 1,000개의 단어를 보존 
    X = vectorizer.fit_transform(pd_list[0])
    
    #TF-IDF 행렬 이용해서 코사인 유사도 계산
    cosine_sim = linear_kernel(X, X)
    
    #csv파일 만들기 위해 pandas 라이브러리의 DataFrame타입 이용
    cosine_df = pd.DataFrame(cosine_sim, index=list_name, columns=list_name)

    #csv로 내보내기
#     cosine_df.to_csv('cosine_df.csv')

    return cosine_df

In [5]:
Novel_name = """he.txt
unnamable.txt
vault.txt
what_moon_brings.txt
whisperer.txt
white_ship.txt
high_house_mist.txt
hound.txt
hypnos.txt
innsmouth.txt
iranon.txt
juan_romero.txt
kadath.txt
lurking_fear.txt
martins_beach.txt
medusas_coil.txt
memory.txt
moon_bog.txt
mountains_of_madness.txt
nameless.txt
nyarlathotep.txt
old_folk.txt
other_gods.txt
outsider.txt
pickman.txt
picture_house.txt
poetry_and_the_gods.txt
polaris.txt
randolph_carter.txt
rats_walls.txt
reanimator.txt
redhook.txt
sarnath.txt
shadow_out_of_time.txt
shunned_house.txt
silver_key.txt
street.txt
temple.txt
terrible_old_man.txt
tomb.txt
tree.txt
ulthar.txt
under_the_pyramids.txt
alchemist.txt
arthur_jermyn.txt
azathoth.txt
beast.txt
beyond_wall_of_sleep.txt
book.txt
celephais.txt
charles_dexter_ward.txt
clergyman.txt
colour_out_of_space.txt
cool_air.txt
crawling_chaos.txt
cthulhu.txt
dagon.txt
descendent.txt
doorstep.txt
dreams_in_the_witch.txt
dunwich.txt
erich_zann.txt
ex_oblivione.txt
festival.txt
from_beyond.txt
gates_of_silver_key.txt
haunter.txt""".split('\n')

In [6]:
Novel_list = []
for name in Novel_name:
    tag = "../source/" + name
    f = open(tag, "r", encoding="utf-8")
    Novel_list.append(f.read().lower()) #전체 읽기
    f.close()

In [7]:
Novel_cossim = texts2csv_of_cossim(Novel_list, Novel_name)
Novel_cossim.to_csv('cosine_df_2.csv')

In [8]:
for idx in tqdm(range(len(Novel_list))):
    co_csv = text2csv_of_co(Novel_list[idx], Novel_name[idx])
    co_csv.to_csv('../csv2/' + Novel_name[idx])

  0%|          | 0/67 [00:00<?, ?it/s]